In [1]:
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset import EEGDataset
from torch.utils.data import random_split

C:\Users\Aksel\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Aksel\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# load in the dataset

raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'

ds1 = EEGDataset(raw_data_dir,2, 250) #Instantiate a dataset using the directory of data, amount of night to include and amount of samples in a segment

//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_1\artefact_annotations.npy
Lables for night 0 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_1\EEG_raw_250hz.npy
Night 0 data loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_2\artefact_annotations.npy
Lables for night 1 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_2\EEG_raw_250hz.npy
Night 1 data loaded


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(250, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

model = NeuralNetwork()

#Weighting artefacts 10x
#loss = nn.BCELoss(weight = torch.tensor([0.1,0.9]))

#weights = [1,10]
#class_weights = torch.FloatTensor(weights)#.cuda()

#loss = nn.CrossEntropyLoss(weight=class_weights) # Using cross entropy and not binary cross entropy because weight implementation is better :/
#loss = nn.CrossEntropyLoss(class_weights)

#pos_weight: amount of positive examples compared to negative examples. Calculate as: negative_examples/positive_examples
loss = nn.BCEWithLogitsLoss(pos_weight = 20*torch.ones([batch_size])) 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X).reshape(-1)# Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
        loss = loss_fn(pred, y.type(torch.FloatTensor))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X).reshape(-1) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor)).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
# Split data into train and test data
trainSamples = int(ds1.__len__()*0.7)
testSamples = int(ds1.__len__() - trainSamples)
training_data, test_data = random_split(ds1, (trainSamples,testSamples), generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [7]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)

print("Done!")

Epoch 1
-------------------------------
loss: 1.011110  [    0/906674]
loss: 1.516259  [ 6400/906674]
loss: 1.533805  [12800/906674]
loss: 1.735191  [19200/906674]
loss: 1.525024  [25600/906674]
loss: 1.525022  [32000/906674]
loss: 1.739555  [38400/906674]
loss: 1.332367  [44800/906674]
loss: 0.907675  [51200/906674]
loss: 1.680017  [57600/906674]
loss: 1.327969  [64000/906674]
loss: 1.327968  [70400/906674]
loss: 1.735143  [76800/906674]
loss: 1.524995  [83200/906674]
loss: 0.903292  [89600/906674]
loss: 1.113432  [96000/906674]
loss: 1.332296  [102400/906674]
loss: 1.800310  [108800/906674]
loss: 1.122139  [115200/906674]
loss: 1.323555  [121600/906674]
loss: 0.706215  [128000/906674]
loss: 1.456937  [134400/906674]
loss: 1.730742  [140800/906674]
loss: 1.940869  [147200/906674]
loss: 1.743788  [153600/906674]
loss: 1.520608  [160000/906674]
loss: 1.323542  [166400/906674]
loss: 1.109049  [172800/906674]
loss: 1.735069  [179200/906674]
loss: 1.113387  [185600/906674]
loss: 1.327841  

RuntimeError: The size of tensor a (64) must match the size of tensor b (50) at non-singleton dimension 0

In [ ]:
print('debug')